# 1. Read the data including the test set

In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', None)



In [37]:
data_all = pd.read_csv('data/all.csv', index_col=False)


25

In [44]:
len(data_all.columns)

25

In [29]:
# df_test = pd.read_csv('data/test.csv', index_col=False)

In [30]:
# df = pd.read_csv('data/train.csv', index_col=False)

In [31]:
# df.info()
# df.head()

In [101]:
data_train = pd.read_csv('data/train1.csv', index_col=False)
# data_train.info()
# data_train.head()

In [33]:
# df_test.info()
# df_test.head()

In [34]:
data_test = data_all[1521787:]
# data_test.info()
# data_test.head()

# 2. Start adding hidden variables

# 2.1 5mins, diff country -- fraud_rate -- add internet score improve 
Since this is the timeframe with different country which have the highest score, going to first filter thru the index and add the 0.2369 to these rows.

TRAIN

In [250]:
# time_and_country_train = data_train[['date', 'time', 'country','city', 'time1', 'hr', 'card_no','money_spent']]
time_country_train = data_train.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent','fraud',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
time_country_train['count'].value_counts()

1    1521787
Name: count, dtype: int64

In [252]:
five_mins_and_diff_country = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_train)-1)):
    if (time_country_train.date.iloc[i+1] == time_country_train.date.iloc[i])\
    & (time_country_train.card_no.iloc[i+1] == time_country_train.card_no.iloc[i])\
    & (int(time_country_train.time.iloc[i+1]) - int(time_country_train.time.iloc[i]) <= 500)\
    & (time_country_train.online_transaction.iloc[i+1] == time_country_train.online_transaction.iloc[i] == 1)\
    & (time_country_train.country.iloc[i+1] != time_country_train.country.iloc[i]):
        five_mins_and_diff_country.append(i)
        five_mins_and_diff_country.append(i+1)

In [255]:
in_five_mins = time_country_train.iloc[five_mins_and_diff_country]
in_five_mins.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
115,8,62.0,183549,18:35:49,18,38,6580,0.00,0,3348,110612,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,742935,1372173,1
116,8,62.0,183924,18:39:24,18,102,5817,591.90,0,6769,110612,5,62,1,8,0.0,0.0,5,0,0,247,82174,0,0,347704,1486449,1
2149,408,85.0,115753,11:57:53,11,104,6364,0.00,0,3655,115235,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,777231,381361,1
2150,408,85.0,115755,11:57:55,11,32,6012,1047.71,0,3271,115235,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,777232,910149,1
2682,472,36.0,30217,03:02:17,3,38,6580,0.00,0,3348,130104,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,700158,354244,1


In [256]:
five_mins_and_diff_country_2 = []

for i in tqdm(range(0, len(time_country_train)-1)):
    if (time_country_train.card_no.iloc[i+1] == time_country_train.card_no.iloc[i])\
    & (time_country_train.date.iloc[i+1] - time_country_train.date.iloc[i] == 1)\
    & (235500<=(int(time_country_train.time.iloc[i]) - int(time_country_train.time.iloc[i+1]))<=235959)\
    & (time_country_train.online_transaction.iloc[i+1] == time_country_train.online_transaction.iloc[i] == 1)\
    & (time_country_train.country.iloc[i+1] != time_country_train.country.iloc[i]):
        five_mins_and_diff_country_2.append(i)
        five_mins_and_diff_country_2.append(i+1)

In [257]:
in_five_mins2 = time_country_train.iloc[five_mins_and_diff_country_2]
in_five_mins2.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
375336,56847,34.0,235958,23:59:58,23,38,621,1.38,0,3288,74708,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,696114,1332406,1
375337,56847,35.0,55,00:00:55,0,46,4356,0.00,0,3351,74708,5,60,1,8,0.0,0.0,5,0,0,296,10891,0,2,698369,439586,1
575629,85860,30.0,235935,23:59:35,23,104,654,0.00,0,3221,123756,5,61,1,8,0.0,0.0,5,0,0,349,42843,0,0,689840,435777,1
575630,85860,31.0,22,00:00:22,0,38,333,2246.07,1,3337,123756,5,60,1,8,0.0,0.0,5,0,0,286,54834,0,0,689862,655440,1
931209,133604,61.0,235705,23:57:05,23,102,5817,1655.75,0,6386,88624,5,62,1,2,0.0,0.0,5,0,0,59,74042,0,0,1898270,804062,1


In [258]:
in_five_mins3 = pd.concat([in_five_mins, in_five_mins2])
in_five_mins3.drop_duplicates(keep='first', inplace=True)
in_five_mins3.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
115,8,62.0,183549,18:35:49,18,38,6580,0.00,0,3348,110612,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,742935,1372173,1
116,8,62.0,183924,18:39:24,18,102,5817,591.90,0,6769,110612,5,62,1,8,0.0,0.0,5,0,0,247,82174,0,0,347704,1486449,1
2149,408,85.0,115753,11:57:53,11,104,6364,0.00,0,3655,115235,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,777231,381361,1
2150,408,85.0,115755,11:57:55,11,32,6012,1047.71,0,3271,115235,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,777232,910149,1
2682,472,36.0,30217,03:02:17,3,38,6580,0.00,0,3348,130104,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,700158,354244,1


In [259]:
len(in_five_mins3)

6591

In [260]:
in_five_mins3.fraud.sum()/len(in_five_mins3.fraud)

0.2369898346229707

In [274]:
list_in_five_mins3 = list(in_five_mins3.index)

In [275]:
time_country_train['in_five_mins'] = 0

In [276]:
time_country_train.in_five_mins.iloc[list_in_five_mins3] = 0.237

In [277]:
time_country_train[time_country_train.in_five_mins== 0.237]

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count,in_five_mins
115,8,62.0,183549,18:35:49,18,38,6580,0.00,0,3348,110612,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,742935,1372173,1,0.237
116,8,62.0,183924,18:39:24,18,102,5817,591.90,0,6769,110612,5,62,1,8,0.0,0.0,5,0,0,247,82174,0,0,347704,1486449,1,0.237
2149,408,85.0,115753,11:57:53,11,104,6364,0.00,0,3655,115235,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,777231,381361,1,0.237
2150,408,85.0,115755,11:57:55,11,32,6012,1047.71,0,3271,115235,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,777232,910149,1,0.237
2682,472,36.0,30217,03:02:17,3,38,6580,0.00,0,3348,130104,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,700158,354244,1,0.237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520578,212765,62.0,5130,00:51:30,0,38,6012,1857.82,0,3271,60805,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,741783,1420760,1,0.237
1520804,212852,60.0,172401,17:24:01,17,38,6580,696.35,0,6215,63334,5,62,1,2,0.0,0.0,5,0,0,380,54238,0,0,1896880,1244294,1,0.237
1520805,212852,60.0,172619,17:26:19,17,46,3460,730.66,0,6215,63334,5,62,1,2,0.0,0.0,5,0,0,289,54828,0,0,1896881,1241729,1,0.237
1521040,212916,84.0,190602,19:06:02,19,104,654,0.00,0,3221,102928,5,61,1,8,0.0,0.0,5,0,0,349,42843,0,0,776292,251434,1,0.237


In [278]:
time_country_train.in_five_mins.value_counts()

0.000    1515196
0.237       6591
Name: in_five_mins, dtype: int64

TEST

In [262]:
time_country_test = data_test.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
time_country_test['count'].value_counts()

1    421665
Name: count, dtype: int64

In [263]:
five_mins_and_diff_country_test = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_test)-1)):
    if (time_country_test.date.iloc[i+1] == time_country_test.date.iloc[i])\
    & (time_country_test.card_no.iloc[i+1] == time_country_test.card_no.iloc[i])\
    & (int(time_country_test.time.iloc[i+1]) - int(time_country_test.time.iloc[i]) <= 500)\
    & (time_country_test.online_transaction.iloc[i+1] == time_country_test.online_transaction.iloc[i] == 1)\
    & (time_country_test.country.iloc[i+1] != time_country_test.country.iloc[i]):
        five_mins_and_diff_country_test.append(i)
        five_mins_and_diff_country_test.append(i+1)

In [264]:
in_five_mins_test = time_country_test.iloc[five_mins_and_diff_country_test]
in_five_mins_test.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
133,76,103.0,170021,17:00:21,17,104,6012,0.00,3655,125947,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,803194,1705570,1
134,76,103.0,170024,17:00:24,17,32,6012,1445.82,3271,125947,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,803195,1705577,1
796,426,102.0,173746,17:37:46,17,104,2800,0.00,3655,107246,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,801755,1861629,1
797,426,102.0,173753,17:37:53,17,32,2800,1557.96,3271,107246,5,62,1,8,0.0,0.0,5,0,0,203,73650,0,0,801756,1861613,1
1933,965,106.0,174227,17:42:27,17,27,3150,853.25,3491,65922,5,71,1,8,0.0,0.0,5,0,0,296,100247,0,2,807370,1538074,1


In [265]:
five_mins_and_diff_country_test_2 = []

for i in tqdm(range(0, len(time_country_test)-1)):
    if (time_country_test.card_no.iloc[i+1] == time_country_test.card_no.iloc[i])\
    & (time_country_test.date.iloc[i+1] - time_country_test.date.iloc[i] == 1)\
    & (235500<=(int(time_country_test.time.iloc[i]) - int(time_country_test.time.iloc[i+1]))<=235959)\
    & (time_country_test.online_transaction.iloc[i+1] == time_country_test.online_transaction.iloc[i] == 1)\
    & (time_country_test.country.iloc[i+1] != time_country_test.country.iloc[i]):
        five_mins_and_diff_country_test_2.append(i)
        five_mins_and_diff_country_test_2.append(i+1)

In [266]:
in_five_mins_test2 = time_country_test.iloc[five_mins_and_diff_country_test_2]
in_five_mins_test2.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
69187,36310,106.0,235809,23:58:09,23,42,621,1962.88,3289,154294,5,62,1,8,0.0,0.0,5,0,0,380,20987,0,2,807798,1546043,1
69188,36310,107.0,0,00:00:00,0,104,1337,0.00,3221,154294,5,61,1,8,0.0,0.0,5,0,0,289,45597,0,2,807874,1546048,1


In [267]:
in_five_mins_test3 = pd.concat([in_five_mins_test, in_five_mins_test2])
in_five_mins_test3.drop_duplicates(keep='first', inplace=True)
in_five_mins_test3.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count
133,76,103.0,170021,17:00:21,17,104,6012,0.00,3655,125947,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,803194,1705570,1
134,76,103.0,170024,17:00:24,17,32,6012,1445.82,3271,125947,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,803195,1705577,1
796,426,102.0,173746,17:37:46,17,104,2800,0.00,3655,107246,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,801755,1861629,1
797,426,102.0,173753,17:37:53,17,32,2800,1557.96,3271,107246,5,62,1,8,0.0,0.0,5,0,0,203,73650,0,0,801756,1861613,1
1933,965,106.0,174227,17:42:27,17,27,3150,853.25,3491,65922,5,71,1,8,0.0,0.0,5,0,0,296,100247,0,2,807370,1538074,1


In [268]:
len(in_five_mins_test3)

2004

In [269]:
list_in_five_mins_test3 = list(in_five_mins_test3.index)

In [270]:
time_country_test['in_five_mins'] = 0

In [271]:
time_country_test.in_five_mins.iloc[list_in_five_mins_test3] = 0.237

In [339]:
time_country_test[time_country_test.in_five_mins ==  0.237]

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,count,in_five_mins
133,76,103.0,170021,17:00:21,17,104,6012,0.00,3655,125947,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,803194,1705570,1,0.237
134,76,103.0,170024,17:00:24,17,32,6012,1445.82,3271,125947,5,62,1,8,0.0,0.0,5,0,0,203,73652,0,0,803195,1705577,1,0.237
796,426,102.0,173746,17:37:46,17,104,2800,0.00,3655,107246,5,62,1,8,0.0,0.0,5,0,0,203,74386,0,0,801755,1861629,1,0.237
797,426,102.0,173753,17:37:53,17,32,2800,1557.96,3271,107246,5,62,1,8,0.0,0.0,5,0,0,203,73650,0,0,801756,1861613,1,0.237
1933,965,106.0,174227,17:42:27,17,27,3150,853.25,3491,65922,5,71,1,8,0.0,0.0,5,0,0,296,100247,0,2,807370,1538074,1,0.237
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420420,213225,117.0,92856,09:28:56,9,93,5729,1037.19,5931,10648,5,62,1,8,0.0,0.0,5,0,0,203,55412,0,0,821572,1691149,1,0.237
420744,213329,92.0,154539,15:45:39,15,38,668,1017.48,5575,155962,5,62,1,8,0.0,0.0,5,0,0,408,33774,0,2,788310,1746805,1,0.237
420745,213329,92.0,154540,15:45:40,15,27,3150,1017.48,3429,155962,5,62,1,8,0.0,0.0,5,0,0,296,72070,0,2,788311,1746794,1,0.237
421648,213564,119.0,150004,15:00:04,15,102,5817,1039.77,6322,122515,5,62,1,8,0.0,0.0,5,0,0,209,38206,0,0,1260523,1528668,1,0.237


In [272]:
time_country_test.in_five_mins.value_counts()

0.000    419661
0.237      2004
Name: in_five_mins, dtype: int64

# 2.2 Differnt country, continue 2+ same money_spent -- 0.163

In [279]:
# time_and_country_train = data_train[['date', 'time', 'country','city', 'time1', 'hr', 'card_no','money_spent']]
time_country_train1 = time_country_train.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent','fraud',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index','in_five_mins']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
# time_country_train1.info()

In [282]:
diff_country_same_money_spent_cont2_day = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_train1)-1)):
    if (time_country_train1.card_no.iloc[i+1] == time_country_train1.card_no.iloc[i] == time_country_train1.card_no.iloc[i-1])\
    & (time_country_train1.country.iloc[i-1] != time_country_train1.country.iloc[i] == time_country_train1.country.iloc[i+1])\
    & (time_country_train1.online_transaction.iloc[i+1] == time_country_train1.online_transaction.iloc[i] == 1)\
    & (time_country_train1.money_spent.iloc[i] == time_country_train1.money_spent.iloc[i+1])\
    & ((time_country_train1.date.iloc[i+1] == time_country_train1.date.iloc[i] == time_country_train1.date.iloc[i-1])\
       or (time_country_train1.date.iloc[i+1] == time_country_train1.date.iloc[i] != time_country_train1.date.iloc[i-1])\
       or (time_country_train1.date.iloc[i+1] != time_country_train1.date.iloc[i] == time_country_train1.date.iloc[i-1])):
        diff_country_same_money_spent_cont2_day.append(i)
        diff_country_same_money_spent_cont2_day.append(i+1)
        

In [283]:
diff_country_cont2 = time_country_train1.iloc[diff_country_same_money_spent_cont2_day]

In [337]:
diff_country_cont2.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,count
624,102,69.0,11914,01:19:14,1,75,1803,2077.32,1,5720,160485,5,20,1,8,0.0,0.0,5,0,0,203,67656,0,2,753440,366488,0.0,1
625,102,69.0,11914,01:19:14,1,75,1803,2077.32,1,5720,160485,5,20,1,8,0.0,0.0,5,0,0,349,67619,0,2,753439,297765,0.0,1
1206,263,89.0,125256,12:52:56,12,75,1803,1335.49,0,5720,17589,5,4,1,8,0.0,0.0,5,0,0,349,67619,0,2,783257,1299175,0.0,1
1207,263,89.0,125257,12:52:57,12,75,1803,1335.49,0,5720,17589,5,4,1,8,0.0,0.0,5,0,0,203,67656,0,2,783258,1301553,0.0,1
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.0,1


In [285]:
diff_country_cont2.fraud.sum()/len(diff_country_cont2.fraud)

0.21809718437783832

In [286]:
len(diff_country_cont2.fraud)

8808

In [288]:
list_diff_country_cont2 = list(diff_country_cont2.index)

In [289]:
time_country_train1['diff_country_cont2'] = 0

In [290]:
time_country_train1.diff_country_cont2.iloc[list_diff_country_cont2] = 0.218

In [338]:
time_country_train1[time_country_train1.diff_country_cont2 == 0.218]

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,count,diff_country_cont2
624,102,69.0,11914,01:19:14,1,75,1803,2077.32,1,5720,160485,5,20,1,8,0.0,0.0,5,0,0,203,67656,0,2,753440,366488,0.0,1,0.218
625,102,69.0,11914,01:19:14,1,75,1803,2077.32,1,5720,160485,5,20,1,8,0.0,0.0,5,0,0,349,67619,0,2,753439,297765,0.0,1,0.218
1206,263,89.0,125256,12:52:56,12,75,1803,1335.49,0,5720,17589,5,4,1,8,0.0,0.0,5,0,0,349,67619,0,2,783257,1299175,0.0,1,0.218
1207,263,89.0,125257,12:52:57,12,75,1803,1335.49,0,5720,17589,5,4,1,8,0.0,0.0,5,0,0,203,67656,0,2,783258,1301553,0.0,1,0.218
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.0,1,0.218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520879,212852,85.0,193543,19:35:43,19,38,6580,236.43,0,6215,63334,5,62,1,2,0.0,0.0,5,0,0,380,54238,0,0,1922183,1363598,0.0,1,0.218
1521411,213072,78.0,124218,12:42:18,12,93,621,632.22,0,3288,129767,5,62,1,8,0.0,0.0,5,0,0,408,20984,0,0,767580,245317,0.0,1,0.218
1521412,213072,78.0,124521,12:45:21,12,93,621,632.22,0,3288,129767,5,62,1,8,0.0,0.0,5,0,0,408,20984,0,0,767583,1478364,0.0,1,0.218
1521469,213092,71.0,203412,20:34:12,20,102,5817,796.90,0,6322,42141,5,62,1,8,0.0,1.0,5,0,0,296,38209,0,0,1590946,1475055,0.0,1,0.218


In [292]:
time_country_train1.diff_country_cont2.value_counts()

0.000    1512979
0.218       8808
Name: diff_country_cont2, dtype: int64

TEST

In [293]:
time_country_test1 = time_country_test.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index','in_five_mins']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
time_country_test['count'].value_counts()

1    421665
Name: count, dtype: int64

In [294]:
diff_country_same_money_spent_cont2_day_test = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_test1)-1)):
    if (time_country_test1.card_no.iloc[i+1] == time_country_test1.card_no.iloc[i] == time_country_test1.card_no.iloc[i-1])\
    & (time_country_test1.country.iloc[i-1] != time_country_test1.country.iloc[i] == time_country_test1.country.iloc[i+1])\
    & (time_country_test1.online_transaction.iloc[i+1] == time_country_test1.online_transaction.iloc[i] == 1)\
    & (time_country_test1.money_spent.iloc[i] == time_country_test1.money_spent.iloc[i+1])\
    & ((time_country_test1.date.iloc[i+1] == time_country_test1.date.iloc[i] == time_country_test1.date.iloc[i-1])\
       or (time_country_test1.date.iloc[i+1] == time_country_test1.date.iloc[i] != time_country_test1.date.iloc[i-1])\
       or (time_country_test1.date.iloc[i+1] != time_country_test1.date.iloc[i] == time_country_test1.date.iloc[i-1])):
        diff_country_same_money_spent_cont2_day_test.append(i)
        diff_country_same_money_spent_cont2_day_test.append(i+1)
        

In [295]:
diff_country_cont2_test = time_country_test1.iloc[diff_country_same_money_spent_cont2_day_test]

In [304]:
diff_country_cont2_test.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,count
1929,965,101.0,175514,17:55:14,17,102,5817,149.14,5975,65922,5,62,1,8,0.0,1.0,5,0,0,209,89980,0,0,965052,1538078,0.000,1
1930,965,101.0,180628,18:06:28,18,102,5817,149.14,5975,65922,5,62,1,8,0.0,1.0,5,0,0,209,89980,0,0,893614,1538077,0.000,1
1932,965,106.0,174039,17:40:39,17,27,3323,853.25,5720,65922,5,71,1,8,0.0,0.0,5,0,0,296,67681,0,0,807367,1538072,0.000,1
1933,965,106.0,174227,17:42:27,17,27,3150,853.25,3491,65922,5,71,1,8,0.0,0.0,5,0,0,296,100247,0,2,807370,1538074,0.237,1
2413,1193,112.0,224657,22:46:57,22,102,5817,513.80,6767,32687,5,62,1,8,0.0,0.0,5,0,0,376,20424,0,0,1685394,1537694,0.000,1


In [297]:
list_diff_country_cont2_test = list(diff_country_cont2_test.index)

In [298]:
len(list_diff_country_cont2_test)

2230

In [299]:
time_country_test1['diff_country_cont2'] = 0

In [300]:
time_country_test1.diff_country_cont2.iloc[list_diff_country_cont2_test] = 0.218

In [303]:
time_country_test1[time_country_test1.diff_country_cont2 == 0.218]

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,count,diff_country_cont2
1929,965,101.0,175514,17:55:14,17,102,5817,149.14,5975,65922,5,62,1,8,0.0,1.0,5,0,0,209,89980,0,0,965052,1538078,0.000,1,0.218
1930,965,101.0,180628,18:06:28,18,102,5817,149.14,5975,65922,5,62,1,8,0.0,1.0,5,0,0,209,89980,0,0,893614,1538077,0.000,1,0.218
1932,965,106.0,174039,17:40:39,17,27,3323,853.25,5720,65922,5,71,1,8,0.0,0.0,5,0,0,296,67681,0,0,807367,1538072,0.000,1,0.218
1933,965,106.0,174227,17:42:27,17,27,3150,853.25,3491,65922,5,71,1,8,0.0,0.0,5,0,0,296,100247,0,2,807370,1538074,0.237,1,0.218
2413,1193,112.0,224657,22:46:57,22,102,5817,513.80,6767,32687,5,62,1,8,0.0,0.0,5,0,0,376,20424,0,0,1685394,1537694,0.000,1,0.218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421160,213423,107.0,191653,19:16:53,19,46,3460,555.43,3348,114391,5,62,1,8,0.0,0.0,5,0,0,289,54828,0,0,808865,1636094,0.000,1,0.218
421164,213423,110.0,4416,00:44:16,0,38,6580,696.35,3348,114391,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,811986,1636086,0.000,1,0.218
421165,213423,110.0,63841,06:38:41,6,38,6580,696.35,3348,114391,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,812175,1636098,0.000,1,0.218
421179,213425,109.0,101312,10:13:12,10,102,5817,1.38,6032,4672,5,75,1,2,0.0,0.0,6,0,0,373,79246,0,0,445401,1743940,0.000,1,0.218


# 2.3 In certain timeframe, different country, 2+ same amount money_spent

In [305]:
# time_and_country_train = data_train[['date', 'time', 'country','city', 'time1', 'hr', 'card_no','money_spent']]
time_country_train2 = time_country_train1.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent','fraud',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index','in_five_mins','diff_country_cont2']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
# time_country_train1.info()

In [306]:
diff_mins_and_diff_country_cont2_1 = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_train2)-1)):
    if (time_country_train2.card_no.iloc[i+1] == time_country_train2.card_no.iloc[i] == time_country_train2.card_no.iloc[i-1])\
    & (time_country_train2.country.iloc[i+1] == time_country_train2.country.iloc[i] != time_country_train2.country.iloc[i-1])\
    & (time_country_train2.money_spent.iloc[i] == time_country_train2.money_spent.iloc[i+1])\
    & (time_country_train2.date.iloc[i+1] == time_country_train2.date.iloc[i] == time_country_train2.date.iloc[i-1])\
    & (int(time_country_train2.time.iloc[i+1]) - int(time_country_train2.time.iloc[i]) <= 500):
        diff_mins_and_diff_country_cont2_1.append(i)
        diff_mins_and_diff_country_cont2_1.append(i+1)
        

In [307]:
diff_mins_and_diff_country_cont2_2 = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_train2)-1)):
    if (time_country_train2.card_no.iloc[i+1] == time_country_train2.card_no.iloc[i] == time_country_train2.card_no.iloc[i-1])\
    & (time_country_train2.country.iloc[i+1] == time_country_train2.country.iloc[i] != time_country_train2.country.iloc[i-1])\
    & (time_country_train2.money_spent.iloc[i] == time_country_train2.money_spent.iloc[i+1])\
    & (time_country_train2.date.iloc[i+1] == time_country_train2.date.iloc[i] == time_country_train2.date.iloc[i-1])\
    & (235500<=(int(time_country_train2.time.iloc[i]) - int(time_country_train2.time.iloc[i+1]))<=235959):
        diff_mins_and_diff_country_cont2_2.append(i)
        diff_mins_and_diff_country_cont2_2.append(i+1)
        

In [308]:
five_mins_and_diff_country_cont2_1 = time_country_train2.iloc[diff_mins_and_diff_country_cont2_1]
five_mins_and_diff_country_cont2_1.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.000,0.218,1
2081,398,42.0,215533,21:55:33,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711027,1412339,0.000,0.218,1
6288,906,45.0,200244,20:02:44,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881343,603920,0.000,0.218,1
6289,906,45.0,200245,20:02:45,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881344,1457562,0.000,0.218,1
10329,1396,34.0,215240,21:52:40,21,93,621,1.38,0,3288,79486,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,698210,1474835,0.237,0.218,1


In [309]:
five_mins_and_diff_country_cont2_2 = time_country_train2.iloc[diff_mins_and_diff_country_cont2_2]
five_mins_and_diff_country_cont2_2.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count


In [310]:
five_mins_and_diff_country_cont2 = pd.concat([five_mins_and_diff_country_cont2_1, five_mins_and_diff_country_cont2_2])
five_mins_and_diff_country_cont2.drop_duplicates(keep='first', inplace=True)
five_mins_and_diff_country_cont2.head(15)

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.000,0.218,1
2081,398,42.0,215533,21:55:33,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711027,1412339,0.000,0.218,1
6288,906,45.0,200244,20:02:44,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881343,603920,0.000,0.218,1
6289,906,45.0,200245,20:02:45,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881344,1457562,0.000,0.218,1
10329,1396,34.0,215240,21:52:40,21,93,621,1.38,0,3288,79486,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,698210,1474835,0.237,0.218,1
10330,1396,34.0,215240,21:52:40,21,93,621,1.38,0,3288,79486,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,698211,1191836,0.237,0.218,1
12382,1652,67.0,133357,13:33:57,13,75,1803,1462.90,0,6760,107073,5,20,0,6,0.0,0.0,5,0,0,349,66655,0,2,1904352,1252339,0.000,0.000,1
12383,1652,67.0,133357,13:33:57,13,75,1803,1462.90,1,6760,107073,5,20,0,2,0.0,0.0,5,0,0,349,66655,0,2,1904353,677837,0.000,0.000,1
12598,1683,73.0,232717,23:27:17,23,104,6580,949.14,0,6540,130659,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1910864,1461174,0.000,0.218,1
12599,1683,73.0,232718,23:27:18,23,104,2224,949.14,0,6001,130659,5,61,1,2,0.0,0.0,5,0,0,296,54215,0,2,1910865,105199,0.000,0.218,1


In [311]:
five_mins_and_diff_country_cont2.fraud.sum()/len(five_mins_and_diff_country_cont2)

0.2980225988700565

In [312]:
len(five_mins_and_diff_country_cont2)

2832

In [313]:
list_five_mins_and_diff_country_cont2 = list(five_mins_and_diff_country_cont2.index)

In [314]:
five_mins_diff_country_cont2 = time_country_train2.iloc[list_five_mins_and_diff_country_cont2]

In [315]:
five_mins_diff_country_cont2.head()

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.000,0.218,1
2081,398,42.0,215533,21:55:33,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711027,1412339,0.000,0.218,1
6288,906,45.0,200244,20:02:44,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881343,603920,0.000,0.218,1
6289,906,45.0,200245,20:02:45,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881344,1457562,0.000,0.218,1
10329,1396,34.0,215240,21:52:40,21,93,621,1.38,0,3288,79486,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,698210,1474835,0.237,0.218,1


In [316]:
len(five_mins_diff_country_cont2)

2832

In [317]:
time_country_train2['five_mins_diff_country_cont2'] = 0

In [318]:
time_country_train2.five_mins_diff_country_cont2.iloc[list_five_mins_and_diff_country_cont2] = 0.298

In [319]:
time_country_train2[time_country_train2.five_mins_diff_country_cont2 == 0.298]

,card_no,date,time,time1,hr,country,city,money_spent,fraud,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count,five_mins_diff_country_cont2
2080,398,42.0,215349,21:53:49,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711092,26470,0.000,0.218,1,0.298
2081,398,42.0,215533,21:55:33,21,38,6580,106.08,0,3348,59441,5,62,1,8,0.0,0.0,5,0,0,380,54238,0,0,711027,1412339,0.000,0.218,1,0.298
6288,906,45.0,200244,20:02:44,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881343,603920,0.000,0.218,1,0.298
6289,906,45.0,200245,20:02:45,20,104,6580,945.48,1,6540,149262,5,61,1,2,0.0,0.0,5,0,0,288,55807,0,2,1881344,1457562,0.000,0.218,1,0.298
10329,1396,34.0,215240,21:52:40,21,93,621,1.38,0,3288,79486,5,62,1,8,0.0,0.0,5,0,0,337,20984,0,2,698210,1474835,0.237,0.218,1,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1520879,212852,85.0,193543,19:35:43,19,38,6580,236.43,0,6215,63334,5,62,1,2,0.0,0.0,5,0,0,380,54238,0,0,1922183,1363598,0.000,0.218,1,0.298
1521639,213196,78.0,120818,12:08:18,12,104,0,0.00,0,0,31612,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1957894,639665,0.000,0.000,1,0.298
1521640,213196,78.0,120830,12:08:30,12,104,0,0.00,0,0,31612,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1957895,556375,0.000,0.000,1,0.298
1521649,213197,80.0,200030,20:00:30,20,104,0,0.00,0,0,31612,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1957954,1300579,0.000,0.000,1,0.298


TEST

In [320]:
time_country_test2 = time_country_test1.groupby(['card_no','date','time','time1', 'hr','country','city','money_spent',\
                                        'receiver_code', 'bank_no','transaction_category', 'currency', 'online_transaction',\
                                         'transaction_type', 'fallback_mark', '3ds_mark', 'payment_type','term_mark', \
                                         'terms','mcc', 'merchandise_code','over_limit', 'situation_code', 'transaction_no',\
                                         'num_index','in_five_mins','diff_country_cont2']).size().to_frame(name='count').reset_index()
# Later have to drop the count column
time_country_test['count'].value_counts()

1    421665
Name: count, dtype: int64

In [321]:
diff_mins_and_diff_country_cont2_1_test = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_test2)-1)):
    if (time_country_test2.card_no.iloc[i+1] == time_country_test2.card_no.iloc[i] == time_country_test2.card_no.iloc[i-1])\
    & (time_country_test2.country.iloc[i+1] == time_country_test2.country.iloc[i] != time_country_test2.country.iloc[i-1])\
    & (time_country_test2.money_spent.iloc[i] == time_country_test2.money_spent.iloc[i+1])\
    & (time_country_test2.date.iloc[i+1] == time_country_test2.date.iloc[i] == time_country_test2.date.iloc[i-1])\
    & (int(time_country_test2.time.iloc[i+1]) - int(time_country_test2.time.iloc[i]) <= 500):
        diff_mins_and_diff_country_cont2_1_test.append(i)
        diff_mins_and_diff_country_cont2_1_test.append(i+1)
        

In [322]:
diff_mins_and_diff_country_cont2_2_test = []

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
for i in tqdm(range(0, len(time_country_test2)-1)):
    if (time_country_test2.card_no.iloc[i+1] == time_country_test2.card_no.iloc[i] == time_country_test2.card_no.iloc[i-1])\
    & (time_country_test2.country.iloc[i+1] == time_country_test2.country.iloc[i] != time_country_test2.country.iloc[i-1])\
    & (time_country_test2.money_spent.iloc[i] == time_country_test2.money_spent.iloc[i+1])\
    & (time_country_test2.date.iloc[i+1] == time_country_test2.date.iloc[i] == time_country_test2.date.iloc[i-1])\
    & (235500<=(int(time_country_test2.time.iloc[i]) - int(time_country_test2.time.iloc[i+1]))<=235959):
        diff_mins_and_diff_country_cont2_2_test.append(i)
        diff_mins_and_diff_country_cont2_2_test.append(i+1)
        

In [323]:
five_mins_diff_country_cont2_1_test = time_country_test2.iloc[diff_mins_and_diff_country_cont2_1_test]
five_mins_diff_country_cont2_1_test.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count
7831,3802,113.0,20304,02:03:04,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816085,1530772,0.237,0.218,1
7832,3802,113.0,20319,02:03:19,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816086,1530770,0.000,0.218,1
11497,5565,99.0,2817,00:28:17,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935029,1749288,0.000,0.218,1
11498,5565,99.0,3035,00:30:35,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935030,1749296,0.000,0.218,1
12667,6262,105.0,113918,11:39:18,11,104,0,0.00,0,23796,5,75,0,1,0.0,0.0,6,0,0,340,103106,0,0,1958580,1643184,0.000,0.000,1


In [324]:
five_mins_diff_country_cont2_2_test = time_country_test2.iloc[diff_mins_and_diff_country_cont2_2_test]
five_mins_diff_country_cont2_2_test.head()

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count


In [325]:
five_mins_diff_country_cont2_test = pd.concat([five_mins_diff_country_cont2_1_test, five_mins_diff_country_cont2_2_test])
five_mins_diff_country_cont2_test.drop_duplicates(keep='first', inplace=True)
five_mins_diff_country_cont2_test.head(15)

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count
7831,3802,113.0,20304,02:03:04,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816085,1530772,0.237,0.218,1
7832,3802,113.0,20319,02:03:19,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816086,1530770,0.000,0.218,1
11497,5565,99.0,2817,00:28:17,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935029,1749288,0.000,0.218,1
11498,5565,99.0,3035,00:30:35,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935030,1749296,0.000,0.218,1
12667,6262,105.0,113918,11:39:18,11,104,0,0.00,0,23796,5,75,0,1,0.0,0.0,6,0,0,340,103106,0,0,1958580,1643184,0.000,0.000,1
12668,6262,105.0,114059,11:40:59,11,104,0,0.00,0,23796,5,75,0,1,0.0,0.0,6,0,0,340,103106,0,0,1958581,1643189,0.000,0.000,1
13702,6677,117.0,204821,20:48:21,20,85,3792,273.83,5575,101220,5,61,1,8,0.0,0.0,5,0,0,288,37417,0,0,822369,1523511,0.237,0.218,1
13703,6677,117.0,204838,20:48:38,20,85,3792,273.83,5575,101220,5,61,1,8,0.0,0.0,5,0,0,288,37417,0,0,822361,1523522,0.237,0.218,1
23513,11892,118.0,193455,19:34:55,19,104,0,0.00,0,123534,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1958950,1801547,0.000,0.000,1
23514,11892,118.0,193557,19:35:57,19,104,0,0.00,0,123534,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1958951,1801530,0.000,0.000,1


In [326]:
list_five_mins_diff_country_cont2_test = list(five_mins_diff_country_cont2_test.index)

In [327]:
len(five_mins_diff_country_cont2_test)

760

In [328]:
time_country_test2['five_mins_diff_country_cont2'] = 0

In [329]:
time_country_test2.five_mins_diff_country_cont2.iloc[list_five_mins_diff_country_cont2_test] = 0.298

In [330]:
time_country_test2[time_country_test2.five_mins_diff_country_cont2 == 0.298]

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count,five_mins_diff_country_cont2
7831,3802,113.0,20304,02:03:04,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816085,1530772,0.237,0.218,1,0.298
7832,3802,113.0,20319,02:03:19,2,104,6580,957.49,3221,95499,5,61,1,8,0.0,0.0,5,0,0,288,55807,0,0,816086,1530770,0.000,0.218,1,0.298
11497,5565,99.0,2817,00:28:17,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935029,1749288,0.000,0.218,1,0.298
11498,5565,99.0,3035,00:30:35,0,75,1689,1104.02,6546,37002,5,62,1,2,0.0,1.0,5,0,0,203,54129,0,2,1935030,1749296,0.000,0.218,1,0.298
12667,6262,105.0,113918,11:39:18,11,104,0,0.00,0,23796,5,75,0,1,0.0,0.0,6,0,0,340,103106,0,0,1958580,1643184,0.000,0.000,1,0.298
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419902,213014,95.0,172829,17:28:29,17,104,0,0.00,0,104504,5,0,0,1,0.0,0.0,1,0,0,340,103106,0,0,1958335,1922973,0.000,0.000,1,0.298
420745,213329,92.0,154540,15:45:40,15,27,3150,1017.48,3429,155962,5,62,1,8,0.0,0.0,5,0,0,296,72070,0,2,788311,1746794,0.237,0.218,1,0.298
420746,213329,92.0,154643,15:46:43,15,27,3150,1017.48,3271,155962,5,62,1,8,0.0,0.0,5,0,0,296,102125,0,2,788314,1746807,0.000,0.218,1,0.298
421467,213497,110.0,231818,23:18:18,23,36,4822,0.00,6734,106603,5,71,0,2,0.0,0.0,5,0,0,380,59471,0,0,1945624,1523789,0.000,0.000,1,0.298


# Save both train and test into new files preparing for ML

In [331]:
time_country_train2.to_csv('data/train2.csv', index=False)

In [332]:
time_country_test2.to_csv('data/test2.csv', index=False)

In [333]:
train2 = pd.read_csv('data/train2.csv', index_col=False)

In [334]:
test2 = pd.read_csv('data/test2.csv', index_col=False)

In [340]:
train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1521787 entries, 0 to 1521786
Data columns (total 30 columns):
card_no                         1521787 non-null int64
date                            1521787 non-null float64
time                            1521787 non-null int64
time1                           1521787 non-null object
hr                              1521787 non-null int64
country                         1521787 non-null int64
city                            1521787 non-null int64
money_spent                     1521787 non-null float64
fraud                           1521787 non-null int64
receiver_code                   1521787 non-null int64
bank_no                         1521787 non-null int64
transaction_category            1521787 non-null int64
currency                        1521787 non-null int64
online_transaction              1521787 non-null int64
transaction_type                1521787 non-null int64
fallback_mark                   1521787 non-null float64


In [341]:
test2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421665 entries, 0 to 421664
Data columns (total 29 columns):
card_no                         421665 non-null int64
date                            421665 non-null float64
time                            421665 non-null int64
time1                           421665 non-null object
hr                              421665 non-null int64
country                         421665 non-null int64
city                            421665 non-null int64
money_spent                     421665 non-null float64
receiver_code                   421665 non-null int64
bank_no                         421665 non-null int64
transaction_category            421665 non-null int64
currency                        421665 non-null int64
online_transaction              421665 non-null int64
transaction_type                421665 non-null int64
fallback_mark                   421665 non-null float64
3ds_mark                        421665 non-null float64
payment_type    

# Which one is better for ML, higher score best less samples or more sampleslower score -- precision and recall probelm!! Which one you value more.

# 2.4 Mcc, mchno

In [237]:
mcc_mchno_train = time_country_train2.groupby(['mcc','merchandise_code']).merchandise_code.count().to_frame(name='merchandise_code_count_in_this_mcc').reset_index()
mcc_mchno_train

,mcc,merchandise_code,merchandise_code_count_in_this_mcc
0,0,38864,4
1,0,43368,1
2,0,86677,1
3,0,91446,1
4,1,58733,1
...,...,...,...
90315,459,2464,51
90316,459,2465,56
90317,459,2466,196
90318,459,2467,163


In [239]:
mcc_mchno_train_fraud = time_country_train2.groupby(['mcc','merchandise_code']).fraud.sum().to_frame(name='fraud_count_in_this_mcc').reset_index()
mcc_mchno_train_fraud

,mcc,merchandise_code,fraud_count_in_this_mcc
0,0,38864,0
1,0,43368,0
2,0,86677,0
3,0,91446,0
4,1,58733,0
...,...,...,...
90315,459,2464,0
90316,459,2465,0
90317,459,2466,0
90318,459,2467,0


In [240]:
mcc_mchno_train['fraud_count_in_this_mcc'] = mcc_mchno_train_fraud.fraud_count_in_this_mcc

In [243]:
mcc_mchno_train['rate_of_fraud_for_each_mchno'] = (mcc_mchno_train.fraud_count_in_this_mcc / mcc_mchno_train.merchandise_code_count_in_this_mcc)

In [346]:
mcc_mchno = mcc_mchno_train.drop(['merchandise_code_count_in_this_mcc','fraud_count_in_this_mcc'], axis=1)

In [352]:
mcc_mchno[(mcc_mchno.mcc==453) & (mcc_mchno.merchandise_code==544)]

,mcc,merchandise_code,rate_of_fraud_for_each_mchno
88394,453,544,0.000257


In [354]:
train3 = pd.merge(time_country_train2, mcc_mchno, on=["mcc", 'merchandise_code'])

In [383]:
train3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1521787 entries, 0 to 1521786
Data columns (total 31 columns):
card_no                         1521787 non-null int64
date                            1521787 non-null float64
time                            1521787 non-null int64
time1                           1521787 non-null object
hr                              1521787 non-null int64
country                         1521787 non-null int64
city                            1521787 non-null int64
money_spent                     1521787 non-null float64
fraud                           1521787 non-null int64
receiver_code                   1521787 non-null int64
bank_no                         1521787 non-null int64
transaction_category            1521787 non-null int64
currency                        1521787 non-null int64
online_transaction              1521787 non-null int64
transaction_type                1521787 non-null int64
fallback_mark                   1521787 non-null float64


In [367]:
test3 = pd.merge(test2, mcc_mchno, on=["mcc", 'merchandise_code'], how='left')

In [375]:
test3.rate_of_fraud_for_each_mchno.fillna(0, inplace=True)

In [378]:
test3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421665 entries, 0 to 421664
Data columns (total 30 columns):
card_no                         421665 non-null int64
date                            421665 non-null float64
time                            421665 non-null int64
time1                           421665 non-null object
hr                              421665 non-null int64
country                         421665 non-null int64
city                            421665 non-null int64
money_spent                     421665 non-null float64
receiver_code                   421665 non-null int64
bank_no                         421665 non-null int64
transaction_category            421665 non-null int64
currency                        421665 non-null int64
online_transaction              421665 non-null int64
transaction_type                421665 non-null int64
fallback_mark                   421665 non-null float64
3ds_mark                        421665 non-null float64
payment_type    

In [350]:
test3[test3.rate_of_fraud_for_each_mchno != 0]

,card_no,date,time,time1,hr,country,city,money_spent,receiver_code,bank_no,transaction_category,currency,online_transaction,transaction_type,fallback_mark,3ds_mark,payment_type,term_mark,terms,mcc,merchandise_code,over_limit,situation_code,transaction_no,num_index,in_five_mins,diff_country_cont2,count,five_mins_diff_country_cont2,rate_of_fraud_for_each_mchno
255,12,97.0,212824,21:28:24,21,102,0,513.80,5103,12731,5,0,0,0,0.0,0.0,5,0,0,453,544,0,0,635272,1616973,0.0,0.0,1,0.0,0.000257
256,12,105.0,214609,21:46:09,21,102,0,513.80,5103,12731,5,0,0,0,0.0,0.0,5,0,0,453,544,0,0,636643,1616974,0.0,0.0,1,0.0,0.000257
257,12,111.0,112522,11:25:22,11,102,0,513.80,4750,12731,5,0,0,0,0.0,0.0,5,0,0,453,544,0,0,637579,1616975,0.0,0.0,1,0.0,0.000257
258,416,117.0,120934,12:09:34,12,102,0,513.80,4274,132340,5,0,0,0,0.0,0.0,5,0,0,453,544,0,0,638669,1539859,0.0,0.0,1,0.0,0.000257
259,538,98.0,4237,00:42:37,0,102,0,513.80,1809,119977,5,0,0,0,0.0,0.0,5,0,0,453,544,0,0,635296,1919828,0.0,0.0,1,0.0,0.000257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400719,207345,107.0,233235,23:32:35,23,102,5817,513.80,5975,97371,5,62,0,4,0.0,0.0,5,0,0,263,98845,0,0,968202,1782691,0.0,0.0,1,0.0,0.136364
400839,208428,104.0,25427,02:54:27,2,16,6167,862.56,3470,49229,5,10,1,8,0.0,0.0,5,0,0,207,21753,0,0,803819,1850599,0.0,0.0,1,0.0,1.000000
400858,208636,94.0,95526,09:55:26,9,46,2690,0.00,5575,118654,5,61,1,8,0.0,0.0,5,0,0,210,39550,0,0,790525,1583597,0.0,0.0,1,0.0,0.250000
401088,211091,112.0,12254,01:22:54,1,104,6001,602.53,2639,79074,5,61,0,2,0.0,0.0,5,0,0,43,103225,0,0,814826,1788250,0.0,0.0,1,0.0,0.200000


Double for loop is not going to work here, merge is the right method to use.

In [245]:
time_country_train2['rate_of_fraud_for_each_mchno'] = 0

1/90320 [01:55<1477:49:38, 58.90s/it]

In [384]:
# for i in tqdm(range(0, len(mcc_mchno_train))):
#     for j in range(0, len(time_country_train2)):
#         if (mcc_mchno_train.mcc.iloc[i] == time_country_train2.mcc.iloc[j]) &\
#             (mcc_mchno_train.merchandise_code.iloc[i] == time_country_train2.merchandise_code.iloc[j]):
#             time_country_train2['rate_of_fraud_for_each_mchno'].iloc[j] = mcc_mchno_train.rate_of_fraud_for_each_mchno.iloc[i]

In [379]:
train4 = train3.drop('count', axis=1)

In [380]:
test4 = test3.drop('count', axis=1)

In [381]:
train4.to_csv('data/train4.csv', index=False)

In [382]:
test4.to_csv('data/test4.csv', index=False)